Import libraries

In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer





Load data and investigate format

In [2]:
pd.set_option('display.max_columns', None)

data = pd.read_csv('cover_data.csv')
print(data.head())
print(data.dtypes)


   Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0       2596      51      3                               258   
1       2590      56      2                               212   
2       2804     139      9                               268   
3       2785     155     18                               242   
4       2595      45      2                               153   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                               0                              510   
1                              -6                              390   
2                              65                             3180   
3                             118                             3090   
4                              -1                              391   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0            221             232            148   
1            220             235            151   
2            234             238   

Split the data into features and labels

Split the data into test and training sets

Standardize and normalize the data so that the continuous features are in a range between 0 and 1

In [3]:
features = data.iloc[:, :-1]
labels = data.iloc[:, -1]


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=4)
numeric_features = features.iloc[:,0:10]
numeric_columns = numeric_features.columns
print(numeric_columns)
ct = ColumnTransformer([('numeric', StandardScaler(), numeric_columns)], remainder='passthrough')
train_features_scaled = ct.fit_transform(train_features)
test_features_scaled = ct.transform(test_features)
train_labels -= 1 # This is necessary to scale the labels down from 1 through 7 to 0 through 6
test_labels -= 1

train_labels_categorical = tf.keras.utils.to_categorical(train_labels)
test_labels_categorical = tf.keras.utils.to_categorical(test_labels)



train_features_df = pd.DataFrame(train_features_scaled)
print(train_features_df)
train_labels_df = pd.DataFrame(train_labels)
print(train_labels_df)
print(train_labels_df['class'].unique())



Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')
              0         1         2         3         4         5         6   \
0       0.312707  0.619846 -0.815006  2.963868  1.959906  2.120101 -0.117131   
1       0.462580 -1.372209 -0.815006  0.288065 -0.351184 -1.196002 -0.117131   
2      -0.761383  1.566743  0.119103 -0.412629  0.179511 -0.595463 -1.088018   
3       0.587474  1.307686  1.720433  0.283362  0.966993  0.256316 -2.843083   
4       1.236924  1.504212 -0.281229 -0.168092 -0.282707 -0.690319 -0.863967   
...          ...       ...       ...       ...       ...       ...       ...   
435754  0.865810  1.021831  0.119103 -0.549006 -0.334064  0.180046 -1.274727   
435755  0.323412 -1.309678  0.519436  1.637723 -0.950355  1.107453 -0.602575   
43

Build a model

In [4]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=train_features_scaled.shape[1]))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])

model.fit(train_features_scaled, train_labels_categorical, epochs=50, batch_size=1000, verbose=1)

model.evaluate(test_features_scaled, test_labels_categorical)



Epoch 1/50


436/436 [==============================] - 1s 2ms/step - loss: 0.7021 - accuracy: 0.7139 - auc: 0.9570
Epoch 2/50
436/436 [==============================] - 1s 2ms/step - loss: 0.5319 - accuracy: 0.7743 - auc: 0.9733
Epoch 3/50
436/436 [==============================] - 1s 2ms/step - loss: 0.4859 - accuracy: 0.7936 - auc: 0.9777
Epoch 4/50
436/436 [==============================] - 1s 2ms/step - loss: 0.4555 - accuracy: 0.8071 - auc: 0.9803
Epoch 5/50
436/436 [==============================] - 1s 2ms/step - loss: 0.4323 - accuracy: 0.8181 - auc: 0.9822
Epoch 6/50
436/436 [==============================] - 1s 2ms/step - loss: 0.4148 - accuracy: 0.8266 - auc: 0.9836
Epoch 7/50
436/436 [==============================] - 1s 3ms/step - loss: 0.4003 - accuracy: 0.8337 - auc: 0.9847
Epoch 8/50
436/436 [==============================] - 1s 2ms/step - loss: 0.3874 - accuracy: 0.8393 - auc: 0.9857
Epoch 9/50
436/436 [==============================] - 1s 2ms/step - loss: 0.3770 - a

[0.2696808874607086, 0.8923946619033813, 0.9924922585487366]